# A Refresher on Production Planning and Inventory Control

## Table of Content <a class="anchor" id="toc"></a>
* [Single-Period Production Model](#single)
* [Multi-Period Production Model](#multi)
* [Multi-Period Production Smoothing Model](#multi_smooth)

## i) Single-Period Production Model <a class="anchor" id="signle"></a> 
[Click here to TOC](#toc)

This is example 2.3-4 from [Operations Research, 8th Edition by Taha](https://www.amazon.com/Operations-Research-Introduction-Hamdy-Taha/dp/0131889230/ref=sr_1_1?ie=UTF8&qid=1537392218&sr=8-1&keywords=operations+research+taha+8).

In preparation for the winter season, a clothing company is manufacturing parka and goose overcoats, insulated pants, and gloves. All products are manufactured in four different departments: cutting, insulating, sewing, and packaging. The company has received firm orders for its products. The contract stipulates a penalty for undelivered items. Devise an optimal production plan for the company based on the following data:

<img src="./single_period.png" style="width: 600px"><br/>

### 1. Model 

The definition of the variables is straightforward. 
- Let $x_1$, $x_2$, $x_3$, $x_4$ be the number of parka jackets, goose jackets, pairs of pants and gloves respectively.
- Let $s_j$ be the number of shortage units of product $j, j = 1, 2, 3, 4$
- Total profit is $30 x_1 + 40 x_2 + 20 x_3 + 10 x_4 - 15 s_1 - 20 s_2 - 10 s_3 - 8 s_4$

So the whole formulation looks like:
$$ \text{max}_{x_j, s_j} \enspace 30 x_1 + 40 x_2 + 20 x_3 + 10 x_4 - 15 s_1 - 20 s_2 - 10 s_3 - 8 s_4 $$ 
$$ \\ 0.3 x_1 + 0.3 x_2 + 0.25 x_3 + 0.15 x_4 \leq 1000 $$ 
$$ \\ 0.25 x_1 + 0.35 x_2 + 0.3 x_3 + 0.1 x_4 \leq 1000 $$
$$ \\ 0.45 x_1 + 0.5 x_2 + 0.4 x_3 + 0.22 x_4 \leq 1000 $$
$$ \\ 0.15 x_1 + 0.15 x_2 + 0.1 x_3 + 0.05 x_4 \leq 1000 $$
$$ \\ x_1 + s_1 = 800 $$
$$ \\ x_2 + s_2 = 750 $$
$$ \\ x_3 + s_3 = 600 $$
$$ \\ x_4 + s_4 = 500 $$
$$ \\ x_j \geq 0, s_j \geq 0, j = 1,2,3,4 $$

### 2. Import Packages

In [1]:
from __future__ import print_function
import sys
import cplex
from cplex.exceptions import CplexError

### 3. Data

In [2]:
import numpy as np

A = [[0.3,  0.3,  0.25, 0.15, 0.0, 0.0, 0.0, 0.0],
     [0.25, 0.35, 0.3,  0.1,  0.0, 0.0, 0.0, 0.0],
     [0.45, 0.5,  0.4,  0.22, 0.0, 0.0, 0.0, 0.0],
     [0.15, 0.15, 0.1,  0.05, 0.0, 0.0, 0.0, 0.0],
     [1.0,  0.0,  0.0,  0.0,  1.0, 0.0, 0.0, 0.0],
     [0.0,  1.0,  0.0,  0.0,  0.0, 1.0, 0.0, 0.0],
     [0.0,  0.0,  1.0,  0.0,  0.0, 0.0, 1.0, 0.0],
     [0.0,  0.0,  0.0,  1.0,  0.0, 0.0, 0.0, 1.0]]
b = [1000.0, 1000.0, 1000.0, 1000.0, 800.0, 750.0, 600.0, 500.0] 
c = [30.0,   40.0,   20.0,   10.0,   -15.0, -20.0, -10.0, -8.0]
u = [cplex.infinity] * len(A[0])
l = [0.0] * len(A[0])
# Optimization objective sense can be 'min' or 'max'
optSense = 'max'  
varType = "C" * len(A[0])
# for senses, G, L, E, R means greater-than, less-than, equality, ranged constraints
senses = ["LLLLEEEE"]
saveOpt = False
saveFile = 'Production Planning'
quietOpt = True

### 4. Code

In [3]:
def mip(A, b, c, u, l, varType, optSense, senses, saveOpt, saveFile, quietOpt):

    noRow, noCol = len(A), len(A[0])
    
    my_prob = cplex.Cplex()
    if optSense == 'min':
        my_prob.objective.set_sense(my_prob.objective.sense.minimize)
    elif optSense == 'max':
        my_prob.objective.set_sense(my_prob.objective.sense.maximize)

    my_colnames = ["x"+str(i) for i in range(noCol)]
    my_prob.variables.add(obj=c, ub=u, lb=l, types=varType, names=my_colnames)
    
    A = [[[i for i in range(noCol)], A[j]] for j in range(noRow)]
    my_prob.linear_constraints.add(lin_expr=A, senses=senses, rhs=b)
    
    if quietOpt:
        my_prob.set_log_stream(None)
        my_prob.set_error_stream(None)
        my_prob.set_warning_stream(None)
        my_prob.set_results_stream(None)
    
    try:
        my_prob.solve()
        if saveOpt:
            my_prob.write( saveFile + ".lp")
            print("LP saved as " + saveFile + ".lp")

        x = my_prob.solution.get_values()
        obj = my_prob.solution.get_objective_value()
        return x, obj
    except CplexError as exc:
        print(exc)
        return [], -1

In [4]:
%%time

x, obj = mip(A, b, c, u, l, varType, optSense, senses, saveOpt, saveFile, quietOpt)
print(x)
print(obj)

[800.0, 750.0, 387.5, 500.0, 0.0, 0.0, 212.5, 0.0]
64625.0
CPU times: user 17.3 ms, sys: 5.09 ms, total: 22.4 ms
Wall time: 20.3 ms


## ii) Multi-Period Production Model <a class="anchor" id="multi"></a> 
[Click here to TOC](#toc)

This is example 2.3-5 from [Operations Research, 8th Edition by Taha](https://www.amazon.com/Operations-Research-Introduction-Hamdy-Taha/dp/0131889230/ref=sr_1_1?ie=UTF8&qid=1537392218&sr=8-1&keywords=operations+research+taha+8).

Acme manufacturing company has contracted to deliver home windows over the next $6$ months. The demands for each month are $100,250,190,140,220$, and $110$ units, respectively. Production cost per window varies from month to month depending on the cost of labour, material, and utilities. Acme estimates the production cost per window over the next 6 months to be $\$50,\$45,\$55,\$48,\$52$, and $\$50$, respectively. To take advantage of the fluctuations in manufacturing cost, Acme may elect to produce more than is needed in a given month and hold the excess units for delivery in later months. This, however, will incur storage costs at the rate of £8 per window per month assessed on end-of-month inventory.

<img src="./multi_period.png" style="width: 600px"><br/>

Develop a linear program to determine the optimum production schedule.

### 1. Model

The variables of the problem include the monthly production amount and the end-of-month inventory. As there are six periods, therefore we can have:<br/>

$$ x_i = \text{Number of units produced in month} \enspace i $$
$$ \\ I_i = \text{Inventory units left at the end of month} \enspace i $$ <br/>
Moreover,
- Assume we don't have any starting inventory. But let's let that be $I_0 = 0$. 
- Production cost is $50 x_1 + 45 x_2 + 55 x_3 + 48 x_4 + 52 x_5 + 50 x_6$
- Inventory cost is $100 I_1 + 250 I_2 + 190 I_3 + 140 I_4 + 220 I_5 + 110 I_6$

So the whole formulation looks like:

$$ min_{x_i, I_i} \enspace 50 x_1 + 45 x_2 + 55 x_3 + 48 x_4 + 52 x_5 + 50 x_6 +8 I_1 + 8 I_2 + 8 I_3 + 8 I_4 + 8 I_5 + 8 I_6 $$
$$ I_0 + x_1 + I_1 = 100 \enspace \text{(1)} \\  $$
$$ I_1 + x_2 + I_2 = 250 \enspace \text{(2)} \\  $$
$$ I_2 + x_3 + I_3 = 190 \enspace \text{(3)} \\  $$
$$ I_3 + x_4 + I_4 = 140 \enspace \text{(4)} \\  $$
$$ I_4 + x_5 + I_5 = 220 \enspace \text{(5)} \\  $$
$$ I_5 + x_6 + I_6 = 110 \enspace \text{(6)} \\  $$
$$ x_i, I_i \geq 0, i = 1,2,3,4,5,6\\  $$

### 2. Data

In [5]:
A = [[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0,  0.0,  0.0,  0.0,  0.0,  0.0],
     [0.0, 1.0, 0.0, 0.0, 0.0, 0.0,  1.0, -1.0,  0.0,  0.0,  0.0,  0.0],
     [0.0, 0.0, 1.0, 0.0, 0.0, 0.0,  0.0,  1.0, -1.0,  0.0,  0.0,  0.0],
     [0.0, 0.0, 0.0, 1.0, 0.0, 0.0,  0.0,  0.0,  1.0, -1.0,  0.0,  0.0],
     [0.0, 0.0, 0.0, 0.0, 1.0, 0.0,  0.0,  0.0,  0.0,  1.0, -1.0,  0.0],
     [0.0, 0.0, 0.0, 0.0, 0.0, 1.0,  0.0,  0.0,  0.0,  0.0,  1.0, -1.0]]
b = [100.0, 250.0, 190.0, 140.0, 220.0, 110.0] 
c = [50.0, 45.0, 55.0, 48.0, 52.0, 50.0] + [8.0] * 6
u = [cplex.infinity] * len(A[0])
l = [0.0] * len(A[0])
# Optimization objective sense can be 'min' or 'max'
optSense = 'min'  
varType = "C" * len(A[0])
# for senses, G, L, E, R means greater-than, less-than, equality, ranged constraints
senses = ["E"] * len(A)
saveOpt = False
saveFile = 'Production Planning'
quietOpt = True

### 3. Code

In [6]:
%%time

x, obj = mip(A, b, c, u, l, varType, optSense, senses, saveOpt, saveFile, quietOpt)
print(x)
print(obj)

[100.0, 440.0, 0.0, 140.0, 220.0, 110.0, 0.0, 190.0, 0.0, 0.0, 0.0, 0.0]
49980.0
CPU times: user 11 ms, sys: 0 ns, total: 11 ms
Wall time: 10.1 ms


Changing the equality sign to inequality sign in constraints (1)-(6) leads us to get the same answer but with faster speed. 

In [7]:
%%time
senses = ["G"] * len(A)
x, obj = mip(A, b, c, u, l, varType, optSense, senses, saveOpt, saveFile, quietOpt)
print(x)
print(obj)

[100.0, 440.0, 0.0, 140.0, 220.0, 110.0, 0.0, 190.0, 0.0, 0.0, 0.0, 0.0]
49980.0
CPU times: user 15 ms, sys: 7.15 ms, total: 22.1 ms
Wall time: 19.1 ms


## iii) Multi-Period Production Smoothing Model <a class="anchor" id="multi_smooth"></a> 
[Click here to TOC](#toc)

This is example 2.3-6 from [Operations Research, 8th Edition by Taha](https://www.amazon.com/Operations-Research-Introduction-Hamdy-Taha/dp/0131889230/ref=sr_1_1?ie=UTF8&qid=1537392218&sr=8-1&keywords=operations+research+taha+8).

The smoothing component of the model originates from the flexibility of employing temps at the start of each time period. 

A company will manufacture a product for the next four months: March, April, May, and June. The demands for each month are $520, 720, 520$, and $620$ units, respectively. The company has a steady workforce of $10$ employees but can meet fluctuating production needs by hiring and firing temporary workers, if necessary. The extra costs of hiring and firing in any month are $\$200$ and $\$400$ per worker, respectively. A permanent worker can produce $12$ units per month, and a temporary worker, lacking comparable experience, only produce $10$ units per month. The company can produce more than needed in any month and carry the surplus over to a succeeding month at a holding cost of $\$50$ per unit per month. 

Develop an optimal hiring/firing policy for the company over the four-month planning horizon.

### 1. Model

There are four stages as specified in the specification. Let's define the variables as:
- $x_i$ is the net number of temps at the start of month $i$ after any hiring or firing
- $S_i^+$ is the number of temps hired at the start of month $i$
- $S_i^-$ is the number of temps fired at the start of month $i$
- $I_i$ is the units of ending inventory for month $i$ 

Therefore,
- Hiring cost = $200 * (S_1^+ + S_2^+ + S_3^+ + S_4^+ )$
- Firing cost = $400 * (S_1^- + S_2^- + S_3^- + S_4^- )$
- Inventory cost = $50 * (I_1 + I_2 + I_3 + I_4)$

So the whole formulation looks like:
$$ \text{min}_{x_i, S_i^+, S_i^-, I_i} \enspace \sum_{i=1}^4 [200 S_i^+ + 400S_i^- + 50 I_i] $$
$$ \\ 10 x_1 - I_1 = 520 - 120 = 400  \enspace \text{(7)}$$
$$ \\ I_1 + 10 x_2 - I_2 = 720 - 120 = 600  \enspace \text{(8)}$$
$$ \\ I_2 + 10 x_3 - I_3 = 520 - 120 = 400  \enspace \text{(9)}$$
$$ \\ I_3 + 10 x_4 - I_4 = 620 - 120 = 500  \enspace \text{(10)}$$
$$ \\ x_1 = S_1^+ - S_1^-        \enspace \Leftrightarrow \enspace \enspace   x_1       - S_1^+ + S_1^ = 0  \enspace \text{(11)} $$
$$ \\ x_2 = x_1 + S_2^+ - S_2^-  \enspace \Leftrightarrow \enspace \enspace - x_1 + x_2 - S_2^+ + S_2^ = 0  \enspace \text{(12)} $$
$$ \\ x_3 = x_2 + S_3^+ - S_3^-  \enspace \Leftrightarrow \enspace \enspace - x_2 + x_3 - S_3^+ + S_3^ = 0  \enspace \text{(13)} $$
$$ \\ x_4 = x_3 + S_4^+ - S_4^-  \enspace \Leftrightarrow \enspace \enspace - x_3 + x_4 - S_4^+ + S_4^ = 0  \enspace \text{(14)} $$
$$ \\ x_i, S_i^+, S_i^-, I_i \geq 0, i=1,2,3,4$$

### 2. Data

In [8]:
A = [[10.0,  0.0,  0.0,  0.0] + [0.0] * 4                + [0.0] * 4            + [-1.0,  0.0,  0.0,  0.0],
     [ 0.0, 10.0,  0.0,  0.0] + [0.0] * 4                + [0.0] * 4            + [ 1.0, -1.0,  0.0,  0.0],
     [ 0.0,  0.0, 10.0,  0.0] + [0.0] * 4                + [0.0] * 4            + [ 0.0,  1.0, -1.0,  0.0],
     [ 0.0,  0.0,  0.0, 10.0] + [0.0] * 4                + [0.0] * 4            + [ 0.0,  0.0,  1.0, -1.0],
     
     [ 1.0,  0.0,  0.0,  0.0] + [-1.0,  0.0,  0.0,  0.0] + [1.0, 0.0, 0.0, 0.0] + [0.0] * 4,
     [-1.0,  1.0,  0.0,  0.0] + [ 0.0, -1.0,  0.0,  0.0] + [0.0, 1.0, 0.0, 0.0] + [0.0] * 4,
     [ 0.0, -1.0,  1.0,  0.0] + [ 0.0,  0.0, -1.0,  0.0] + [0.0, 0.0, 1.0, 0.0] + [0.0] * 4,
     [ 0.0,  0.0, -1.0,  1.0] + [ 0.0,  0.0,  0.0, -1.0] + [0.0, 0.0, 0.0, 1.0] + [0.0] * 4]

b = [400.0, 600.0, 400.0, 500.0] + [0.0] * 4
c = [0.0] * 4 + [200.0] * 4 + [400.0] * 4 + [50.0] * 4
u = [cplex.infinity] * len(A[0])
l = [0.0] * len(A[0])
# Optimization objective sense can be 'min' or 'max'
optSense = 'min'  
varType = "C" * len(A[0])
# for senses, G, L, E, R means greater-than, less-than, equality, ranged constraints
senses = ["E"] * 4 + ["E"] * 4
saveOpt = False
saveFile = 'Production Planning'
quietOpt = True

In [9]:
%%time

x, obj = mip(A, b, c, u, l, varType, optSense, senses, saveOpt, saveFile, quietOpt)
print(x)
print(obj)

[50.0, 50.0, 45.0, 45.0, 50.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 100.0, 0.0, 50.0, 0.0]
19500.0
CPU times: user 2.85 ms, sys: 3.1 ms, total: 5.95 ms
Wall time: 5.38 ms


In the course of debugging, I found that inputting the A matrix is very prone to typos and mistakes. For example, let's say I accidentially typed two of the dots as commas as follows:

<img src="./mistake.png" style="width: 600px"><br/>

In [10]:
A = [[10.0,  0.0,  0.0,  0.0] + [0.0] * 4                + [0.0] * 4            + [-1.0,  0.0,  0.0,  0.0],
     [ 0.0, 10.0,  0.0,  0.0] + [0.0] * 4                + [0.0] * 4            + [ 1.0, -1.0,  0.0,  0.0],
     [ 0.0,  0.0, 10.0,  0.0] + [0.0] * 4                + [0.0] * 4            + [ 0.0,  1.0, -1.0,  0.0],
     [ 0.0,  0.0,  0.0, 10.0] + [0.0] * 4                + [0.0] * 4            + [ 0.0,  0.0,  1.0, -1.0],
     
     [ 1.0,  0.0,  0,0,  0.0] + [-1.0,  0.0,  0.0,  0.0] + [1.0, 0.0, 0.0, 0.0] + [0.0] * 4,
     [-1.0,  1.0,  0,0,  0.0] + [ 0.0, -1.0,  0.0,  0.0] + [0.0, 1.0, 0.0, 0.0] + [0.0] * 4,
     [ 0.0, -1.0,  1.0,  0.0] + [ 0.0,  0.0, -1.0,  0.0] + [0.0, 0.0, 1.0, 0.0] + [0.0] * 4,
     [ 0.0,  0.0, -1.0,  1.0] + [ 0.0,  0.0,  0.0, -1.0] + [0.0, 0.0, 0.0, 1.0] + [0.0] * 4]

We can run the LP but the result is not what we want. Indeed it took me an entire afternoon to debug.

In [11]:
%%time
senses = ["E"] * 4 + ["E"] * 4
x, obj = mip(A, b, c, u, l, varType, optSense, senses, saveOpt, saveFile, quietOpt)
print(x)
print(obj)

[51.666666666666664, 48.333333333333336, 45.0, 45.0, 0.0, 51.666666666666664, 0.0, 0.0, 0.0, 0.0, 3.333333333333333, 0.0, 116.66666666666663, 0.0, 50.0, 0.0]
20000.0
CPU times: user 18.3 ms, sys: 2.6 ms, total: 20.9 ms
Wall time: 18.8 ms


So let's change the way we input our `A` matrix (i.e. specifying the position and value of an element explicitly).

In [12]:
def mip_v2(A, b, c, u, l, varType, optSense, senses, saveOpt, colNames, saveFile, quietOpt):

    noRow, noCol = len(A), len(A[0])
    
    my_prob = cplex.Cplex()
    if optSense == 'min':
        my_prob.objective.set_sense(my_prob.objective.sense.minimize)
    elif optSense == 'max':
        my_prob.objective.set_sense(my_prob.objective.sense.maximize)

    my_prob.variables.add(obj=c, ub=u, lb=l, types=varType, names=colNames)
    
    my_prob.linear_constraints.add(lin_expr=A, senses=senses, rhs=b)
    
    if quietOpt:
        my_prob.set_log_stream(None)
        my_prob.set_error_stream(None)
        my_prob.set_warning_stream(None)
        my_prob.set_results_stream(None)
    
    try:
        my_prob.solve()
        if saveOpt:
            my_prob.write( saveFile + ".lp")
            print("LP saved as " + saveFile + ".lp")

        x = my_prob.solution.get_values()
        obj = my_prob.solution.get_objective_value()
        return x, obj
    except CplexError as exc:
        print(exc)
        return [], -1

In [13]:
colNames = ["x_" + str(i) for i in range(1,5)] \
         + ["S_plus_" + str(i) for i in range(1,5)] \
         + ["S_minus_" + str(i) for i in range(1,5)] \
         + ["I_" + str(i) for i in range(1,5)]
A = [
     [["x_1", "I_1"], [10, -1]],
     [["x_2", "I_1", "I_2"], [10, 1, -1]],
     [["x_3", "I_2", "I_3"], [10, 1, -1]],
     [["x_4", "I_3", "I_4"], [10, 1, -1]],
     [["x_1", "S_plus_1", "S_minus_1"], [1, -1, 1]],
     [["x_1", "x_2", "S_plus_2", "S_minus_1"], [-1, 1, -1, 1]],
     [["x_2", "x_3", "S_plus_3", "S_minus_2"], [-1, 1, -1, 1]],
     [["x_3", "x_4", "S_plus_4", "S_minus_3"], [-1, 1, -1, 1]]
    ]

In the end, we got the same result as before.

In [14]:
%%time
x, obj = mip_v2(A, b, c, u, l, varType, optSense, senses, saveOpt, colNames, saveFile, quietOpt)
print(x)
print(obj)

[50.0, 50.0, 45.0, 45.0, 50.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 100.0, 0.0, 50.0, 0.0]
19500.0
CPU times: user 13.4 ms, sys: 2.03 ms, total: 15.4 ms
Wall time: 14.2 ms


Finally, my two cents on debugging a LP:
- Always try your LP with a small dataset
- Save your LP as a file and see if the printed formulation is what you want
- Try to implement your LP with different solvers (e.g. Cplex, Gurobi, TORA, Lingo) and compare their result